In [13]:
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.2 MB/s eta 0:00:00


In [14]:
import psycopg2
import numpy as np
import pandas as pd

from io import StringIO
from datetime import datetime, timedelta

In [15]:
pd.set_option('max_colwidth', 999999)
pd.options.mode.copy_on_write = True

In [18]:
PG_HOST = "34.101.187.125"
PG_DB = "postgres"
PG_PORT = 5432
PG_USER = "postgres"
PG_PASSWORD = "Kampang06@"

In [16]:
def connect_to_db(PG_USER, PG_PASSWORD, PG_HOST, PG_PORT, PG_DB):
    """
    Parameters
    ----------
    PG_USER : username of Postgres database
    PG_PASSWORD : password of Postgres database
    PG_HOST : host/ip address of Postgres database
    PG_PORT : port of Postgres database
    PG_DB : name of Postgres database

    Returns
    -------
    engine : the starting point for any psycopg2 application
    cursor : provide methods to execute the PostgreSQL commands in the database using python code
    """
    try:
        # Initialize engine object
        print("Connecting to the Postgres database. Please wait...")
        engine = None
        engine = psycopg2.connect(
            user=PG_USER,
            password=PG_PASSWORD,
            host=PG_HOST,
            port=PG_PORT,
            database=PG_DB,
        )

        # Initialize cursor object
        cursor = engine.cursor()
        cursor.execute("SET TIMEZONE TO 'Asia/Jakarta';")  # set timezone
        print(
            "Database connection to Cloud SQL (Postgres) has been successfully initialized."
        )

        # Return engine and cursor object
        return engine, cursor
    # Handle any error and exception
    except Exception as e:
        raise e

In [17]:
def close_file_and_connection(file, engine, cursor):
    """
    Parameters
    ----------
    file : opened file
    engine : opened engine
    cursor : opened cursor

    Returns
    -------
    no returns
    """
    try:
        file.close()
        cursor.close()
        engine.close()
        print(
            "Opened file and database connection has been successfully closed."
        )
    # Handle any error and exception
    except Exception as e:
        raise e

In [20]:
def export_data_from_db():
    """
    Parameters
    ----------
    table_name : name of the table
    time_col : datetime column for filtering data -> created_at

    Returns
    -------
    no returns
    """
    try:
        # Get engine and cursor object from connect_to_db function
        engine, cursor = connect_to_db(PG_USER, PG_PASSWORD, PG_HOST, PG_PORT, PG_DB)
        cursor.execute("SET TIMEZONE TO 'Asia/Jakarta';") # set timezone

        # Define SQL query to extract daily data
        sql_query = f"""
                        SELECT
                        id
                        ,parent_id
                        ,lawyer_id
                        ,client_id
                        ,status
                        ,lawyer_approved
                        ,lawyer_attendance
                        ,client_attendance
                        ,CASE
                            WHEN created_at IS NULL THEN CAST(TO_CHAR(CURRENT_TIMESTAMP, 'YYYY-MM-DD HH24:MI:SS') AS TIMESTAMP)
                            ELSE created_at
                         END AS created_at
                        ,CASE
                            WHEN updated_at IS NULL THEN NULL
                            ELSE updated_at
                         END AS updated_at
                        ,CASE
                            WHEN deleted_at IS NULL THEN NULL
                            ELSE deleted_at
                         END AS deleted_at
                        ,CASE
                            WHEN paid_at IS NULL THEN NULL
                            ELSE paid_at
                         END AS paid_at
                        ,CASE
                            WHEN lawyer_approved_at IS NULL THEN NULL
                            ELSE lawyer_approved_at
                         END AS lawyer_approved_at
                        ,CASE
                            WHEN last_call IS NULL THEN NULL
                            ELSE last_call
                         END AS last_call
                        ,CASE
                            WHEN stop_time IS NULL THEN NULL
                            ELSE stop_time
                         END AS stop_time
                        ,CASE
                            WHEN summary_sent_at IS NULL THEN NULL
                            ELSE summary_sent_at
                         END AS summary_sent_at
                        FROM consultations
                        WHERE created_at >= NOW() - INTERVAL '30 minutes'
                    """
        sql_command = "COPY ({0}) TO STDOUT WITH DELIMITER AS '|' CSV HEADER QUOTE E'\"' ENCODING 'UTF-8'".format(sql_query)

        # Save extracted data to local directory
        with open(f'/content/consultations_20240226_1530.csv', 'w') as file:
            cursor.copy_expert(sql_command, file)

        print(f"30-min data from table name: consultations has been successfully exported to a CSV file.")
    # Handle any error and exception
    except Exception as e:
        raise e
    finally:
        # Close opened file and database connection
        close_file_and_connection(file, engine, cursor)